# Water Lead Data Exploration

## Initial data loads and imports and function definitions

In [1]:
#imports

import pandas as pd
import numpy as np
import sklearn as skl
from urllib2 import Request, urlopen
import json
from sodapy import Socrata
from sklearn.metrics.pairwise import cosine_similarity
from difflib import get_close_matches
from tabula import read_pdf

In [2]:
#Load Data
##### We should not have to run this often. 
#### CODES####
#  water_lead Sampling: ja9d-vnbi
# childhood BLL: d54z-enu8
# student weight status: es3k-2aus

def loadData(nyDataCode):
    client = Socrata("health.data.ny.gov", 'fxYuH4PYiai1VuSoppMr69046')

    results = client.get(nyDataCode, limit=50000)

    results_df = pd.DataFrame.from_records(results)
    return results_df

# school_locations = waterLeadResults[['School District','School', 'County', 'Type of Organization', 'BEDS Code', 'School Zip Code', 'Location', ]]

In [3]:
#filters multi-indexed dataframes
# use: filter_by(df, {'level_name' : value})
def filter_by(df, constraints):
    """Filter MultiIndex by sublevels."""
    indexer = [constraints[name] if name in constraints else slice(None)
               for name in df.index.names]
    return df.loc[tuple(indexer)] if len(df.shape) == 1 else df.loc[tuple(indexer),]

In [4]:
# initial Imports

enrollment = pd.read_csv('PublicSchool2016EconDisadv.csv')
schoolLocationTable = pd.read_csv('schoolLocations.csv')
waterLeadResults = pd.read_csv('Lead_Testing_in_School_Drinking_Water.csv')
# reportCard = pd.read_csv('2017_reportCard.csv')
trimmedReportCard = pd.read_csv('trimmedReportCard.csv')
lead_econ = pd.read_csv('lead_and_Econ.csv')
plumbing = pd.read_csv('Plumbing_constructionAge.csv')

In [5]:
# enrollment[enrollment['COUNTY']=='QUEENS'][['LOCATION NAME','SUBGROUP NAME','GRADE 1', 'GRADE 2', 'GRADE 3','GRADE 4', 'GRADE 5','GRADE 6', 'GRADE 7', 'GRADE 8', 'GRADE 9', 'GRADE 10', 'GRADE 11', 'GRADE 12']]
# enrollment.columns
enrollment[enrollment['LOCATION NAME'].str.contains('JAMAICA')]
# plumbing[plumbing['Zip'] == ]
# lead_only[lead_only['perc outlets over']>.9]


,SCHOOL YEAR,DATE OF REPORT,COUNTY,STATE DISTRICT ID,DISTRICT NAME,STATE LOCATION ID,LOCATION NAME,SCHOOL TYPE,SUBGROUP CODE,SUBGROUP NAME,...,GRADE 5,GRADE 6,UNGRADED (ELEMENTARY),GRADE 7,GRADE 8,GRADE 9,GRADE 10,GRADE 11,GRADE 12,UNGRADED (SECONDARY)
5549,2015-16,4/28/2017,QUEENS,342800010000,NYC GEOG DIST #28 - QUEENS,342800010312,JAMAICA CHILDREN'S SCHOOL,PUBLIC,15,Economically Disadvantaged,...,0,0,0,0,0,0,0,0,0,0
5550,2015-16,4/28/2017,QUEENS,342800010000,NYC GEOG DIST #28 - QUEENS,342800010312,JAMAICA CHILDREN'S SCHOOL,PUBLIC,16,Not Economically Disadvantaged,...,-,-,-,-,-,-,-,-,-,-
5571,2015-16,4/28/2017,QUEENS,342800010000,NYC GEOG DIST #28 - QUEENS,342800011350,JAMAICA GATEWAY TO THE SCIENCES,PUBLIC,15,Economically Disadvantaged,...,0,0,0,0,0,95,106,84,87,0
5572,2015-16,4/28/2017,QUEENS,342800010000,NYC GEOG DIST #28 - QUEENS,342800011350,JAMAICA GATEWAY TO THE SCIENCES,PUBLIC,16,Not Economically Disadvantaged,...,0,0,0,0,0,39,-,36,32,-


# Data Cleaning
## Enrollment - economic Status
Group enrollment by economic status ('Disad' or 'Not Disad') and by subject matter (Math and ELA)
scale Disad and Not Disad separately (i think this is wrong, and we should scale on total school enrollment)

In [6]:
Econ_disadvantaged = enrollment[enrollment['SUBGROUP NAME'] == 'Economically Disadvantaged']
Econ_advantaged = enrollment[enrollment['SUBGROUP NAME'] == 'Not Economically Disadvantaged']


In [7]:
cols = [u'DATE OF REPORT',  u'STATE DISTRICT ID',
       u'DISTRICT NAME', u'STATE LOCATION ID', 
       u'SCHOOL TYPE', u'SUBGROUP CODE', 
       u'PK (HALF DAY)', u'PK (FULL DAY)', u'KG (HALF DAY)', u'KG (FULL DAY)',
       u'GRADE 1', u'GRADE 2', u'GRADE 3', u'GRADE 4', u'GRADE 5', u'GRADE 6',
       u'UNGRADED (ELEMENTARY)', u'GRADE 7', u'GRADE 8', u'GRADE 9',
       u'GRADE 10', u'GRADE 11', u'GRADE 12', u'UNGRADED (SECONDARY)']

for col in cols:
    del Econ_advantaged[col]
    del Econ_disadvantaged[col]

In [8]:
Econ_disadvantaged = Econ_disadvantaged.rename(index=str, columns={"PK12 TOTAL": "DisAd PK12 TOTAL"})
Econ_advantaged = Econ_advantaged.rename(index=str, columns={"PK12 TOTAL": "Adv PK12 TOTAL"})

In [9]:
EconStatus = Econ_disadvantaged.merge(Econ_advantaged, left_on='LOCATION NAME', right_on='LOCATION NAME')

In [10]:
# trimmedReportCard.columns
# trimmedReportCard.loc[(trimmedReportCard['ITEM_DESC'] == 'Grade 3 Math')]
trimmedReportCard['TOTAL_TESTED'] = trimmedReportCard['TOTAL_TESTED'].apply(pd.to_numeric, errors='coerce')
## filter out aggregated rows (beds code ending in '000')
trimmedReportCard['Is School'] = trimmedReportCard['BEDSCODE'].apply(lambda x: not(x%1000 == 0))
trimmedReportCard = trimmedReportCard[trimmedReportCard['Is School'] == True]

schoolGroup = trimmedReportCard.groupby(['BEDSCODE','SUBGROUP_NAME'])
econGrouped = pd.DataFrame(trimmedReportCard.groupby(['BEDSCODE','ITEM_SUBJECT_AREA','SUBGROUP_NAME']).agg({'TOTAL_TESTED': 'sum'}))

In [11]:
## TO DELETE ##
temp = econGrouped[np.in1d(econGrouped.index.get_level_values(2), ['Not Economically Disadvantaged'])]
temp = temp.reset_index()
del temp['SUBGROUP_NAME']
# econStatus = econStatus.merge(right=temp, right_on='Zip', left_on='Zip')  
# del econStatus['Econ Not Disadvantaged']
# econStatus = econStatus.rename(columns={'TOTAL_TESTED_x':'EconDisad', 'TOTAL_TESTED_y':'NotEconDisad'})
# del econStatus['TOTAL_TESTED']

In [12]:
# trimmedReportCard#['Scaled Total Tested'] = scaleColumn(trimmedReportCard['TOTAL_TESTED'])
mathEconStat = econGrouped[np.in1d(econGrouped.index.get_level_values(1), ['Mathematics'])].unstack()
mathEconStat.index = mathEconStat.index.droplevel(1)
mathEconStat = mathEconStat['TOTAL_TESTED']
elaEconStat = econGrouped[np.in1d(econGrouped.index.get_level_values(1), ['ELA'])].unstack()
elaEconStat.index = elaEconStat.index.droplevel(1)
elaEconStat = elaEconStat['TOTAL_TESTED']

In [13]:
mathEconStat['Economically Disadvantaged'] = mathEconStat['Economically Disadvantaged'].fillna(0)
mathEconStat['Not Economically Disadvantaged'] = mathEconStat['Not Economically Disadvantaged'].fillna(0)
mathEconStat['math Econ Dis perc'] = mathEconStat['Economically Disadvantaged']/(mathEconStat['Not Economically Disadvantaged']+mathEconStat['Economically Disadvantaged'])
mathEconStat['math Econ Dis perc'] = mathEconStat['math Econ Dis perc'].fillna(0)
mathEconStat = mathEconStat.reset_index()

elaEconStat['ela Econ Dis perc'] = elaEconStat['Economically Disadvantaged']/(elaEconStat['Not Economically Disadvantaged']+elaEconStat['Economically Disadvantaged'])
elaEconStat = elaEconStat.reset_index()

In [26]:
tmp = waterLeadResults[['School','BEDS Code']]
tmp['BEDS Code'] = pd.to_numeric(tmp['BEDS Code'], errors='coerce')

sample2 = sampleData.merge(tmp, left_on='guess1', right_on='School')
sample2 = sample2.merge(scaledMathEconStat, left_on='BEDS Code', right_on='BEDSCODE')
sample2 = sample2.rename(columns={'scaled Econ Dis perc': 'math Econ Disad'})
sample2 = sample2.merge(scaledElaEconStat, left_on='BEDS Code', right_on='BEDSCODE')
sample2 = sample2.rename(columns={'scaled Econ Dis perc': 'ELA Econ Disad'})
del sample2['BEDSCODE_x']
del sample2['BEDSCODE_y']


/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


NameError: name 'scaledMathEconStat' is not defined

# TO DO

## Rescale economic status using total enrollment at the school

# Data Cleaning

## Age of Plumbing Systems

### Fuzzy Wuzzy Lookup
used fuzzy lookup to get best match between Plumbing age and lead results. Results in 895 schools with "high confidence match" (over 90%). This is the sample set we will use for now. 

Merged these 895 schools into one table including plumbing age and lead results. DF is called "SampledData"


In [16]:
def fuzzyLookup(df, zip_col, lookup_col, lim=2):
    from fuzzywuzzy import fuzz, process
    zips = schoolLocationTable['location_zip'].drop_duplicates()
    zips = zips.dropna()
    NameCheck = pd.DataFrame()
    for z in zips:
        try:
            choices = schoolLocationTable[schoolLocationTable['location_zip'] == str(z)]
            choices = choices['school']
            temp = df[df[zip_col] == int(z)]
            temp['bestGuessSchool'] = temp[lookup_col].apply \
                                    (lambda x: process.extract(x, choices, limit=lim, scorer=fuzz.token_set_ratio))
            NameCheck = NameCheck.append(temp)
        except:
            pass
    
    guesses = []
    for i in range(1,lim+1):
        guesses.append('guess'+str(i))
    NameCheck[guesses] = pd.DataFrame(NameCheck.bestGuessSchool.values.tolist(), index= NameCheck.index)
    del NameCheck['bestGuessSchool']
    
    ### Clean up fuzzyWuzzy results in dataframe 
    for col in guesses:
        name = col + '_score'
        NameCheck[name] = NameCheck[col].apply(lambda x: x[1] if not(x == None) else x)
        NameCheck[col] = NameCheck[col].apply(lambda x: x[0] if not(x == None) else x)
    
    return NameCheck

In [17]:
def fuzzyCombinedID(df):
    ## handling fuzzyWuzzy accuracy lower than 90%
    tmp = df
    tmp = tmp[(tmp['guess1_score'] < 90) & (tmp['guess1_score'] >= 80)]

    ## combine two columns to make "stronger" identifier
    tmp['full_ID'] = tmp['On Demand Identifier'] + ' ' + tmp['Entity Name']

    ## Redo lookup using new identifier
    tmp = fuzzyLookup(df=tmp, zip_col='Zip', lookup_col = 'full_ID', lim=2)
    return tmp


In [18]:
leadCluster = waterLeadResults.merge(schoolLocationTable[['beds_code','lat','long']], left_on='BEDS Code', right_on='beds_code')
leadCluster = leadCluster[['School District','School','County','Number of Outlets','Number of Outlets, Result ≤ 15 ppb','Number of Outlets, Result > 15 ppb','BEDS Code','School Street ','School City','School State','School Zip Code','County Location','Location','beds_code','lat','long']]
leadCluster = leadCluster.rename(columns={'Number of Outlets, Result ≤ 15 ppb':'outlets under 15ppb','Number of Outlets, Result > 15 ppb': 'outlets over 15ppb'})
leadCluster['outlets over 15ppb'] = leadCluster['outlets over 15ppb'].fillna(0)
leadCluster['outlets under 15ppb'] = leadCluster['outlets under 15ppb'].fillna(0)
leadCluster['perc outlets over'] = leadCluster['outlets over 15ppb']/(leadCluster['outlets over 15ppb']+leadCluster['outlets under 15ppb'])
leadCluster['perc outlets over'] = leadCluster['perc outlets over'].fillna(0)
leadCluster

,School District,School,County,Number of Outlets,outlets under 15ppb,outlets over 15ppb,BEDS Code,School Street,School City,School State,School Zip Code,County Location,Location,beds_code,lat,long,perc outlets over
0,BEACON CITY SCHOOLS,BEACON HS,Dutchess,NaN,0.0,0.0,130200010005,101 MATTEAWAN RD,BEACON,NY,12508.0,"(41.686216, -73.840468)","101 MATTEAWAN RD\nBEACON, NY 12508\n(41.513494...",130200010005,41.513495,-73.962103,0.000000
1,SOMERS CENTRAL SCHOOL,SOMERS MIDDLE SCHOOL,Westchester,50.0,0.0,0.0,662101060004,RT 202,SOMERS,NY,10589.0,"(41.039278, -73.805386)","RT 202\nSOMERS, NY 10589\n(41.33904983300005, ...",662101060004,41.339050,-73.673095,0.000000
2,SYOSSET CENTRAL SCHOOL,SOUTH GROVE SCHOOL,Nassau,44.0,0.0,0.0,280502060007,COLONY LN,SYOSSET,NY,11791.0,"(40.715749, -73.601185)","COLONY LN\nSYOSSET, NY 11791\n(40.801028961000...",280502060007,40.801029,-73.507151,0.000000
3,BOCES No District Name,TOMPKINS-SENECA-TIOGA BOCES BUILDING E DARWIN ...,Tompkins,NaN,0.0,0.0,619000000012,555 WARREN ROAD,ITHACA,NY,14850.0,"(42.461024, -76.478784)","555 WARREN ROAD\nITHACA, NY 14850\n(42.4753686...",619000000012,42.475369,-76.466304,0.000000
4,BOCES No District Name,FRANKLIN-ESSEX-HAMILTON BOCES NORTH FRANKLIN E...,Franklin,NaN,0.0,0.0,169000000002,23 HUSKIE LANE,MALONE,NY,12953.0,"(44.705699, -74.340621)","23 HUSKIE LANE\nMALONE, NY 12953\n(44.84126042...",169000000002,44.841260,-74.281553,0.000000
5,TULLY CENTRAL SCHOOL,TULLY J S H S,Onondaga,166.0,130.0,23.0,421902040001,ELM ST,TULLY,NY,13159.0,"(43.065629, -76.168033)","ELM ST\nTULLY, NY 13159\n(42.79780683000007, -...",421902040001,42.797807,-76.111865,0.150327
6,BOCES No District Name,ROCKLAND BOCES OCCUPATIONAL ED CENTER CLASSROO...,Rockland,NaN,0.0,0.0,509000000003,65 PARROTT ROAD,WEST NYACK,NY,10994.0,"(41.127287, -74.017033)","65 PARROTT ROAD\nWEST NYACK, NY 10994\n(41.121...",509000000003,41.121625,-73.980590,0.000000
7,BRIARCLIFF MANOR U F S,BRIARCLIFF MS,Westchester,NaN,0.0,0.0,661402020004,444 PLEASANTVILLE RD,BRIARCLIFF MANOR,NY,10510.0,"(41.039278, -73.805386)","444 PLEASANTVILLE RD\nBRIARCLIFF MANOR, NY 105...",661402020004,41.136405,-73.811562,0.000000
8,REMSEN CENTRAL SCHOOL,REMSEN J S H S,Oneida,83.0,57.0,26.0,411701040002,MAIN ST,REMSEN,NY,13438.0,"(43.149482, -75.361773)","MAIN ST\nREMSEN, NY 13438\n(43.32135975600005,...",411701040002,43.321360,-75.185740,0.313253
9,BOCES No District Name,WESTCHESTER BOCES CAREER CAMPUS BUILDING A (LE...,Westchester,NaN,0.0,0.0,669000000049,65 GRASSLANDS ROAD,VALHALLA,NY,10595.0,"(41.039278, -73.805386)","65 GRASSLANDS ROAD\nVALHALLA, NY 10595\n(41.07...",669000000049,41.070085,-73.784314,0.000000


In [19]:
leadCluster.to_csv('SchoolLeadResults.csv')

In [21]:
## Fuzzy wuzzy matches over 90% accurate are correct
plumbingNameCheck = fuzzyLookup(df=plumbing, zip_col='Zip', lookup_col = 'On Demand Identifier', lim=2)
sampleData = plumbingNameCheck[plumbingNameCheck['guess1_score'] >= 90]

## Fuzzy wuzzy match using second lookup (combined identifier)
nextSchools1 = fuzzyCombinedID(plumbingNameCheck)
sampleData = sampleData.append(nextSchools1[(nextSchools1['guess1_score'] == 100)])

## Fuzzy wuzzy between 80-90% (~525) using manual check for accuracy (241)
nextSchools2 = pd.read_csv('242matchedschools.csv')
sampleData = sampleData.append(nextSchools2)

## drop duplicates in case a school was captured in multiple of the above scenarios
sampleData = sampleData.drop_duplicates()

## trim DataFrame to relevant columns to prepare for data augmentation
sampleData  = sampleData[['Zip','Entity Name', 'On Demand Identifier','guess1', 'age of Distribution System','age of Drainage System','age of Water Heaters','age of plumbing fixtures']]

# Data Cleaning
## Scaling lead data
lead outlets results are scaled across all 4600 data points, even when sampling is on a shorter list. 

In [22]:
# col = waterLeadResults['number_of_outlets_result_greater']

def scaleColumn(array):
    import numpy as np
    from sklearn import preprocessing

    data = np.nan_to_num(array)
    data = data.reshape(-1,1)
    
    scaler = preprocessing.MinMaxScaler()
    scaler.fit(data)
    
    scaledData = scaler.transform(data)
    return scaledData


In [23]:


# bad = shortWaterLead['Number of Outlets, Result > 15 ppb']
# good = shortWaterLead['Number of Outlets, Result \xe2\x89\xa4 15 ppb']


def compareColumns(bad, good):

    both = bad + good
    bad_scaled = scaleColumn(bad)
    good_scaled = scaleColumn(good)
    both_scaled =  bad_scaled + good_scaled

    return bad_scaled/both_scaled

In [24]:
shortWaterLead = waterLeadResults[['School Zip Code', 'BEDS Code', 'School','County','Number of Outlets, Result \xe2\x89\xa4 15 ppb','Number of Outlets, Result > 15 ppb']]

# Data Sampling for clustering analysis



In [25]:
clusterSample = sampleData 
clusterSample = clusterSample.merge(leadCluster,left_on='guess1',right_on='School')
clusterSample[['long','lat','age of Distribution System','scaled Percent over']].to_csv('sampleClustering.csv',index=False)

KeyError: "['scaled Percent over'] not in index"

In [28]:
clusterSample['age of Distribution System'].corr(clusterSample['scaled Percent over'],method='pearson')

NameError: name 'clusterSample' is not defined

In [521]:
clusterSample3feats = sample2 
clusterSample3feats = clusterSample3feats.merge(leadCluster,left_on='guess1',right_on='School')
clusterSample3feats = clusterSample3feats.fillna(0)
clusterSample3feats[['long','lat','age of Distribution System','scaled Percent over','math Econ Disad']].to_csv('sampleClustering_3 features.csv',index=False)

In [606]:
leadCluster['beds_code'] = pd.to_numeric(leadCluster['beds_code'],errors='coerce')
lead_only = leadCluster
lead_econ = leadCluster.merge(mathEconStat, left_on='beds_code', right_on='BEDSCODE')
lead_plumbing = lead_only.merge(sampleData, left_on='School', right_on='guess1')
lead_econ_plumbing = lead_econ.merge(sampleData, left_on='School', right_on='guess1')


In [607]:
lead_only.to_csv('lead_only.csv')
lead_econ.to_csv('lead_econ_sample.csv')
lead_plumbing.to_csv('lead_plumbin_sample.csv')
lead_econ_plumbing.to_csv('lead_econ_plumbing_sample.csv')

## Future Work - EPA web API for contaminant lookup

In [33]:
import pandas as pd

In [48]:
url = "https://iaspub.epa.gov/enviro/efservice/VIOLATION/CONTAMINANT_CODE/5000/WATER_SYSTEM/STATE_CODE/NY/ROWS/0:999/"

In [49]:
import json
import requests

xml_data = requests.get(url).content

In [50]:
import xml.etree.ElementTree as ET
import pandas as pd

class XML2DataFrame:

    def __init__(self, xml_data):
        self.root = ET.XML(xml_data)

    def parse_root(self, root):
        return [self.parse_element(child) for child in iter(root)]

    def parse_element(self, element, parsed=None):
        if parsed is None:
            parsed = dict()
        for key in element.keys():
            parsed[key] = element.attrib.get(key)
        if element.text:
            parsed[element.tag] = element.text
        for child in list(element):
            self.parse_element(child, parsed)
        return parsed

    def process_data(self):
        structure_data = self.parse_root(self.root)
        return pd.DataFrame(structure_data)

xml2df = XML2DataFrame(xml_data)
xml_dataframe = xml2df.process_data()

In [47]:
xml_dataframe

,ADDRESS_LINE1,ADDRESS_LINE2,ADMIN_NAME,ALT_PHONE_NUMBER,CDS_ID,CITIES_SERVED,CITY_NAME,COMPLIANCE_STATUS_CODE,COMPL_PER_BEGIN_DATE,COMPL_PER_END_DATE,...,STATE_MCL,SUBMISSION_STATUS_CODE,VIOLATION,VIOLATION_CATEGORY_CODE,VIOLATION_CODE,VIOLATION_ID,VIOLATION_ROW,VIOL_MEASURE,WATER_SYSTEM,ZIP_CODE
0,SENECA NATION HEALTH DEPT,PO BOX 231,Theresa Lay,NaN,NaN,Not Reported,SALAMANCA,R,01-JAN-06,NaN,...,NaN,Y,\n,MR,52,#06005,\n,NaN,\n,14779
1,SENECA NATION HEALTH DEPT,PO BOX 231,"PATTERSON, PATRICIA",NaN,NaN,Not Reported,SALAMANCA,NaN,NaN,NaN,...,NaN,Y,\n,NaN,NaN,NaN,NaN,NaN,\n,14779
2,NaN,1510 ROUTE 438,IRVING INDUSTRIAL PARK,NaN,NaN,Not Reported,IRVING,NaN,NaN,NaN,...,NaN,Y,\n,NaN,NaN,NaN,NaN,NaN,\n,14081
3,SENECA NATION HEALTH DEPT,PO BOX 231,Theresa Lay,NaN,NaN,Not Reported,SALAMANCA,R,01-JAN-06,NaN,...,NaN,Y,\n,MR,52,#06005,\n,NaN,\n,14779
4,"BOX 8A, COMMUNITY BUILDING",NaN,Ernie Thompson,NaN,NaN,Not Reported,HOGANSBURG,R,01-JAN-15,NaN,...,NaN,Y,\n,MR,52,000000552500012015,\n,NaN,\n,13655
5,SENECA NATION OF INDIANS,P.O. BOX 231,THOMAS INDIAN SCHOOL,NaN,NaN,Not Reported,SALAMANCA,NaN,NaN,NaN,...,NaN,Y,\n,NaN,NaN,NaN,NaN,NaN,\n,14779
6,PO BOX 231,NaN,David Bova,NaN,NaN,Not Reported,SALAMANCA,NaN,NaN,NaN,...,NaN,Y,\n,NaN,NaN,NaN,NaN,NaN,\n,14779
7,SENECA NATION HEALTH DEPT,PO BOX 231,Theresa Lay,NaN,NaN,Not Reported,SALAMANCA,R,01-JAN-13,NaN,...,NaN,Y,\n,MR,52,000000852500012013,\n,NaN,\n,14779
8,BOX 319B,NaN,Irving Powless,NaN,NaN,Not Reported,NEDROW,R,01-JAN-13,NaN,...,NaN,Y,\n,MR,52,000001252500012013,\n,NaN,\n,13120
9,ST. REGIS MOHAWK TRIBE,"BOX 8A, COMMUNITY BUILDING",ST. REGIS MOHAWK TRIBE,NaN,NaN,Not Reported,HOGANSBURG,NaN,NaN,NaN,...,NaN,Y,\n,NaN,NaN,NaN,NaN,NaN,\n,13655
